In [15]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import sklearn.model_selection as ms
import numpy as np
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import *
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import accuracy_score
import plotly.express as px
import pickle

In [16]:
rbData = pd.read_parquet("rbDataV2.parquet")
wrData = pd.read_parquet("wrDataV2.parquet")
teData = pd.read_parquet("teDataV2.parquet")

In [17]:
rbData['position'] = [0 for i in range(rbData.shape[0])]
wrData['position'] = [1 for i in range(wrData.shape[0])]
teData['position'] = [2 for i in range(teData.shape[0])]

In [18]:
df = pd.concat([rbData, wrData, teData])

In [19]:
tier = []
threshold = 12
for row in df.itertuples():
    if row.fantasy_points_ppr_per_game_future >= threshold:
        tier.append(1)
    else:
        tier.append(0)
df['tier_future'] = tier
nextYear = df[df.season == 2021].copy()
df = df.dropna(subset=['fantasy_points_ppr_future'])

In [20]:
colsToDrop = ['player_id', 'season',
              'fantasy_points_ppr_future', 
              'fantasy_points_ppr_per_game_future',
              'fantasy_rank_future',
              'per_game_fantasy_rank_future',
              'tier_future']
# separate featues and labels
features = df.drop(colsToDrop, axis=1)
labels = df['tier_future']

In [23]:
highestMean = 0
bestDepth = 0
bestEst = 0
bestRate = 0
# define model evaluation method
cv = ms.StratifiedKFold(n_splits=15)
for depth in range(2, 11, 2):
    for est in range(50, 251, 50):
        for rate in [0.1, 0.2, 0.3]:
            # fit model no training data
            model = xgb.XGBClassifier(max_depth=depth, n_estimators=est, learning_rate=rate)
            # evaluate model
            scores = ms.cross_val_score(model, features, labels, scoring='accuracy', cv=cv, n_jobs=-1)
            if scores.mean() > highestMean:
                highestMean = scores.mean()
                bestDepth = depth
                bestEst = est
                bestRate = rate
print("best mean:", highestMean)

best mean: 0.875221681779059


In [24]:
optimalModel = xgb.XGBClassifier(max_depth=bestDepth, n_estimators=bestEst, learning_rate=bestRate)

In [25]:
optimalModel.fit(features, labels)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=2, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=50, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [26]:
file = "OverUnder12Model.pkl"
pickle.dump(optimalModel, open(file, "wb"))